# Import libraries

In [1]:
import os
import copy
from tqdm.notebook import tqdm
import pprint
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import torch
from src.constants import DATA_PATH, MASTER_THESIS_DIR, TRAINING_CONFIG_PATH, MODEL_CONFIG_PATH
from src.data_loader.data_set import Data_Set
from src.experiments.utils import get_experiement_args, process_experiment_args
from src.models.baseline_model import BaselineModel
from src.utils import get_console_logger, read_json
from src.visualization.visualize import plot_hand
from torch.utils.data import DataLoader
from torchvision import transforms
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
from src.data_loader.utils import convert_2_5D_to_3D, convert_to_2_5D
from src.experiments.evaluation_utils import calculate_epe_statistics, evaluate,get_predictions_and_ground_truth, cal_auc_joints, get_pck_curves
from src.data_loader.utils import get_train_val_split
from src.models.supervised_head_model import SupervisedHead

# Load configuration

In [2]:
model_config = edict(read_json(os.path.join(MASTER_THESIS_DIR, "src", "experiments","config", "model_config.json")))
joints_mapping = {v: k for k, v in read_json("/local/home/adahiya/Documents/master_thesis/src/data_loader/joint_mapping.json")["ait"].items()}
model_config.gpu =True
train_config =  edict(read_json(TRAINING_CONFIG_PATH))
train_config.augmentation_flags.crop =False
train_config.augmentation_flags.resize =False
train_config.augmentation_flags.rotate=False
train_config.augmentation_flags.cut_out=False
train_config.augmentation_flags.color_jitter=False
train_config.augmentation_flags.gaussian_blur=False
train_config.augmentation_flags.color_drop=False
print(model_config)
train_config.num_workers=8
train_config.epochs = 150
model_config.warmup_epochs = 10
train_config, model_config

{'alpha': 5, 'batch_size': 32, 'gpu': True, 'lr': 1e-05, 'opt_weight_decay': 1e-05, 'resnet_trainable': True, 'warmup_epochs': 10}


({'batch_size': 128,
  'epochs': 150,
  'train_ratio': 0.9,
  'gpu': True,
  'num_workers': 8,
  'seed': 5,
  'augmentation_flags': {'color_drop': False,
   'color_jitter': False,
   'crop': False,
   'cut_out': False,
   'flip': False,
   'gaussian_blur': False,
   'random_crop': False,
   'resize': False,
   'rotate': False},
  'augmentation_order': [],
  'augmentation_flags0': {'color_drop': True,
   'color_jitter': True,
   'crop': True,
   'cut_out': True,
   'flip': False,
   'gaussian_blur': True,
   'random_crop': False,
   'resize': True,
   'rotate': True},
  'augmentation_params': {'crop_margin': 1.25,
   'crop_margin_range': [0.6, 2.0],
   'cut_out_fraction': [0.0, 0.16],
   'hue_factor_range': [0.01, 1.0],
   'max_angle': 359,
   'min_angle': 0,
   'resize_shape': [128, 128],
   'sat_factor_range': [0.01, 1.0],
   'value_factor_alpha_range': [0.5, 1],
   'value_factor_beta_range': [5, 20],
   'crop_box_jitter': [0.0, 1.0]}},
 {'alpha': 5,
  'batch_size': 32,
  'gpu': True,

# Load data

In [3]:
train_data = Data_Set(
    config=train_config,
    transform=transforms.Compose(
        [transforms.ToTensor()]
    ),
    train_set=True,
)
val_data = copy.copy(train_data)
val_data.is_training(False)

##  Sanity check against all zero predictions 
only 3d

In [4]:
joints3d = []
print("Reading the data")
for i in tqdm(range(len(val_data))):
    joints3d.append(val_data[i]["joints3D"])
Joints3d = torch.stack(joints3d)
Joints3d_zero = torch.zeros_like(Joints3d)
stats = calculate_epe_statistics(Joints3d_zero, Joints3d, 3)
print(stats)

Reading the data


KeyboardInterrupt: 

# Load model

In [4]:
# supervised model
model = BaselineModel(config=model_config)
checkpoint = torch.load("/local/home/adahiya/Documents/master_thesis/data/models/master-thesis/a33039073f0346cd9a5b0f42068a07de/checkpoints/epoch=49.ckpt"
)
model.load_state_dict(checkpoint["state_dict"])
model.eval()
device = torch.device("cuda")
model.to(device)
print()

In [6]:
results = evaluate(model, val_data, num_workers=8,batch_size=32)
pprint.pprint(results)
pred_dict = get_predictions_and_ground_truth(model, val_data,num_workers=8,batch_size=32)
pred = pred_dict["predictions_3d"]
gt = pred_dict["ground_truth_3d"]
sts =calculate_epe_statistics(pred, gt, dim=3)
eucledian_dist = sts['eucledian_dist']
y,x = get_pck_curves(eucledian_dist, per_joint=True)
auc = cal_auc_joints(eucledian_dist, per_joint=True)
print(f"AUC : {np.mean(auc)}")

407it [00:08, 50.53it/s]
100%|██████████| 13024/13024 [00:03<00:00, 3964.09it/s]


{'AUC': 0.03732211339774861,
 'Mean_EPE_2D': tensor(88.4632),
 'Mean_EPE_3D': tensor(2.0805),
 'Mean_EPE_3D_R': tensor(2.0928),
 'Mean_EPE_3D_R_v_3D': tensor(0.0328),
 'Median_EPE_2D': tensor(89.4856),
 'Median_EPE_3D': tensor(1.2581),
 'Median_EPE_3D_R': tensor(1.2601),
 'Median_EPE_3D_R_V_3D': tensor(0.0055)}


407it [00:07, 51.02it/s]
100%|██████████| 13024/13024 [00:03<00:00, 3824.87it/s]


AUC : 0.03732211339774861


In [23]:
results = evaluate(model, val_data, num_workers=8,batch_size=32)
pprint.pprint(results)
pred_dict = get_predictions_and_ground_truth(model, val_data,num_workers=8,batch_size=32)
pred = pred_dict["predictions_3d"]
gt = pred_dict["ground_truth_3d"]
sts =calculate_epe_statistics(pred, gt, dim=3)
eucledian_dist = sts['eucledian_dist']
y,x = get_pck_curves(eucledian_dist, per_joint=True)
auc = cal_auc_joints(eucledian_dist, per_joint=True)
print(f"AUC : {np.mean(auc)}")

407it [00:07, 51.88it/s]
100%|██████████| 13024/13024 [00:03<00:00, 3885.01it/s]

{'AUC': 0.0408362531583015,
 'Mean_EPE_2D': tensor(88.4632),
 'Mean_EPE_3D': tensor(2.0741),
 'Mean_EPE_3D_R': tensor(2.0842),
 'Mean_EPE_3D_R_v_3D': tensor(0.0227),
 'Median_EPE_2D': tensor(89.4856),
 'Median_EPE_3D': tensor(1.2526),
 'Median_EPE_3D_R': tensor(1.2525),
 'Median_EPE_3D_R_V_3D': tensor(1.1927e-07)}



407it [00:07, 51.71it/s]
100%|██████████| 13024/13024 [00:03<00:00, 3895.91it/s]

AUC : 0.0408362531583015


In [ ]:
# semi supervised model
model = SupervisedHead(
    model_config,
    "/local/home/adahiya/Documents/master_thesis/data/models/master-thesis/14910753afb9499ab9229174d6377efb/checkpoints/epoch=999.ckpt",
    model_config,
)
checkpoint = torch.load(
    "/local/home/adahiya/Documents/master_thesis/data/models/master-thesis/abf5b702ea4c4ad78703b6f81221acd4/checkpoints/epoch=149.ckpt"
)["state_dict"]
model.load_state_dict(checkpoint)
model.eval()

# calculate Results

In [20]:
# strong augmentations
results = evaluate(model, val_data, num_workers=8,batch_size=32)
pprint.pprint(results)
pred_dict = get_predictions_and_ground_truth(model, val_data,num_workers=8,batch_size=32)
pred = pred_dict["predictions_3d"]
gt = pred_dict["ground_truth_3d"]
sts =calculate_epe_statistics(pred, gt, dim=3)
eucledian_dist = sts['eucledian_dist']
y,x = get_pck_curves(eucledian_dist, per_joint=True)
auc = cal_auc_joints(eucledian_dist, per_joint=True)
print(f"AUC : {np.mean(auc)}")

407it [00:03, 119.32it/s]
100%|██████████| 13024/13024 [00:03<00:00, 4030.42it/s]

{'AUC': 0.585265172025574,
 'Mean_EPE_2D': tensor(5.9256),
 'Mean_EPE_3D': tensor(0.2445),
 'Mean_EPE_3D_R': tensor(0.1633),
 'Mean_EPE_3D_R_v_3D': tensor(0.2039),
 'Median_EPE_2D': tensor(4.2474),
 'Median_EPE_3D': tensor(0.1705),
 'Median_EPE_3D_R': tensor(0.0591),
 'Median_EPE_3D_R_V_3D': tensor(0.1502)}



407it [00:03, 120.02it/s]
100%|██████████| 13024/13024 [00:03<00:00, 3886.08it/s]

AUC : 0.5865171068008422


In [ ]:
# weak augmentations
results = evaluate(model, val_data, num_workers=8,batch_size=32)
pprint.pprint(results)
pred_dict = get_predictions_and_ground_truth(model, val_data,num_workers=8,batch_size=32)
pred = pred_dict["predictions_3d"]
gt = pred_dict["ground_truth_3d"]
sts =calculate_epe_statistics(pred, gt, dim=3)
eucledian_dist = sts['eucledian_dist']
y,x = get_pck_curves(eucledian_dist, per_joint=True)
auc = cal_auc_joints(eucledian_dist, per_joint=True)
print(f"AUC : {np.mean(auc)}")

#  Visualize sample predictions

In [ ]:
model.to(torch.device('cpu'))
print()

In [ ]:
@interact(idx=widgets.IntSlider(min=1, max=100, step=5, value=3),plot_gt= widgets.Checkbox(value=True, description='Ground truth'),
         plot_pred= widgets.Checkbox(value=True, description='Predicted_labels'))
def visualize1(idx, plot_gt, plot_pred):
    sample = val_data[idx]
    joints = sample["joints"]
    img = sample["image"]
    img_input = img.view(([1]+list(img.shape)))
    prediction = model(img_input).view(21,3).detach().numpy()
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    plt.imshow(transforms.ToPILImage()(img))
    if plot_gt:
       plot_hand(ax, joints)
    if plot_pred:
        plot_hand(ax, prediction,linestyle=':' )
    plt.show()

In [ ]:
@interact(joint_id=widgets.IntSlider(min=0, max=20, step=1, value=3))

def visualize2(joint_id):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111)
    plt.plot(x,y[joint_id])
    plt.title(f"PCK curve, AUC: {auc[joint_id]}")
    plt.plot(x,np.mean(y,axis=0), color='black',linestyle=":" ,alpha=0.5)
    plt.xlabel("error in mm")
    plt.ylabel("Ratio of points below the error")
    plt.legend([f"PCK curve for {joints_mapping[joint_id]}", "Average PCK curve"])
    plt.show()